## 读取IMDB文字数据

In [59]:
import keras
import numpy as np
import pandas

df = pandas.read_csv('https://github.com/ywchiu/pydeeplearning/raw/master/data/movie_review.csv', index_col=0)
#df.head()
reviews = ''.join([' <SOR> '+ele+' <EOR> ' for ele in df['content'].head(300).tolist()]).lower()
len(reviews)

607140

## 产生训练用序列与目标

In [60]:
# 训练用序列长度
maxlen = 60

# 随机采样序列
step = 3

sentences = []
next_chars = []

for i in range(0, len(reviews) - maxlen, step):
    # 根据步骤，每次取出maxlen的序列做为特征
    sentences.append(reviews[i: i + maxlen])
    # 取出maxlen + 1的字当目标
    next_chars.append(reviews[i + maxlen])


In [61]:
len(sentences)

202360

## one-hot 编码

In [64]:
# 找出独特字元
chars = sorted(list(set(reviews)))
len(chars)

57

In [66]:
# 利用字典映射char和索引
char_indices = dict((char, chars.index(char)) for char in chars)
#char_indices

In [67]:
# x：训练样本
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)

# y：目标样本
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):        
    for t, char in enumerate(sentence):         
        x[i, t, char_indices[char]] = 1         
    y[i, char_indices[next_chars[i]]] = 1       

In [68]:
x.shape

(202360, 60, 57)

In [69]:
y.shape

(202360, 57)

## 建立LSTM模型

In [73]:
from keras import layers

model = keras.models.Sequential()
# 输入为60 *字串长度
model.add(layers.LSTM(128,input_shape=(maxlen, len(chars))))

# 以字元对应作为输出
model.add(layers.Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

## 训练模型

In [78]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('imdb2.hdf5', monitor='loss', verbose=1, save_best_only=True)

In [79]:
history = model.fit(x, y, batch_size=128, epochs=15, callbacks=[checkpoint])

Epoch 1/15
202360/202360 [==============================] - 125s 618us/step - loss: 1.6877
Epoch 2/15
202360/202360 [==============================] - 135s 670us/step - loss: 1.4175
Epoch 3/15
202360/202360 [==============================] - 141s 696us/step - loss: 1.1390
Epoch 4/15
202360/202360 [==============================] - 145s 717us/step - loss: 0.8918
Epoch 5/15
202360/202360 [==============================] - 144s 711us/step - loss: 0.6941
Epoch 6/15
202360/202360 [==============================] - 144s 713us/step - loss: 0.5484
Epoch 7/15
202360/202360 [==============================] - 145s 717us/step - loss: 0.4444
Epoch 8/15
202360/202360 [==============================] - 145s 717us/step - loss: 0.3626
Epoch 9/15
202360/202360 [==============================] - 145s 719us/step - loss: 0.3021
Epoch 10/15
202360/202360 [==============================] - 146s 721us/step - loss: 0.2551
Epoch 11/15
202360/202360 [==============================] - 146s 723us/step - loss: 0.22

In [80]:
%ls 

4-Copy1.6 Training LSTM Model.ipynb     4.6 Training LSTM Model.ipynb
4.4 Stock Prediction.ipynb              imdb.hdf5
4.5 Retrieve IMDB User's Reviews.ipynb  imdb2.hdf5


In [87]:
reviews[7:67]

'rest assured, iron man is an absolutely amazing movie. i won'